In [ ]:
import json
import torch
import torch.nn as nn
import numpy as np

import string





### Preprocessing

In [ ]:
allowed_chars = string.ascii_lowercase + " "

# Load and preprocess the text data
with open("pokemon.txt", "r") as f:
    lines = [line.strip().lower() for line in f.readlines()]

# Filter out unwanted characters
lines = [''.join(c for c in line if c in allowed_chars) for line in lines]
text = "".join(lines)
chars = sorted(list(set(text)))
char_to_int = {char: i for i, char in enumerate(chars)}

int_text = [char_to_int[char] for char in text]




In [ ]:
def one_hot_encode(sequence, num_categories):
    result = torch.zeros((len(sequence), num_categories))
    for i, category in enumerate(sequence):
        result[i, category] = 1
    return result


### Neural network definition

In [ ]:

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden.expand(input.size(0), -1)), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

### Hyperparameters

In [ ]:
input_size = output_size = len(chars)
hidden_size = 1024
model = RNN(input_size, hidden_size, output_size)
loss_fn = nn.NLLLoss()
learning_rate = 0.005
num_epochs = 1000
batch_size = 128
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)




### Training

In [ ]:

for epoch in range(num_epochs):
    hidden = model.init_hidden()
    losses = []
    for i in range(0, len(int_text) - batch_size, batch_size):
        inputs = one_hot_encode(int_text[i:i+batch_size], input_size)
        targets = torch.tensor(int_text[i+1:i+batch_size+1])
        outputs, hidden = model(inputs, hidden.detach())
        loss = loss_fn(outputs, targets)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: loss={np.mean(losses)}")

### Test

In [ ]:

def generate_name(start, name_length):
    generated_name = start
    hidden = model.init_hidden()
    for _ in range(name_length):
        input = one_hot_encode([char_to_int[generated_name[-1]]], input_size)
        output, hidden = model(input, hidden)
        probabilities = torch.exp(output).detach().numpy().flatten()
        next_char = np.random.choice(chars, p=probabilities)
        generated_name += next_char
    return generated_name.capitalize()

num_names = 10
name_length = 10
generated_names = [generate_name('u', name_length) for _ in range(num_names)]
print(generated_names)



### Save the trained model and configuration

In [ ]:
def save_model(model, model_file, config_file):
    torch.save(model.state_dict(), model_file)
    config = {
        "input_size": input_size,
        "hidden_size": hidden_size,
        "output_size": output_size,
    }
    with open(config_file, "w") as f:
        json.dump(config, f)
        
save_model(model, "RNN_pokemon.pth", "RNN_pokemon_config.json")

In [ ]:
from sklearn.model_selection import ParameterGrid
from torch.utils.data import DataLoader

# Set the training parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 100
batch_size = 128

# Define the parameter grid for hyperparameter search
param_grid = {
    "hidden_size": [128, 256],
    "num_layers": [1, 2],
    "learning_rate": [0.01, 0.001],
    "dropout_rate": [0, 0.5],
}

# Create a list to store the results of the hyperparameter search
results = []

# Perform grid search
for params in ParameterGrid(param_grid):
    model = RNN(input_size, params["hidden_size"], output_size, params["num_layers"], params["dropout_rate"]).to(device)
    loss_fn = nn.NLLLoss()
    optimizer = optim.SGD(model.parameters(), lr=params["learning_rate"])
    
    # Create the DataLoader for your training data
    # Replace this with your DataLoader implementation
    train_dataloader = DataLoader(...)
    
    # Train the model
    for epoch in range(num_epochs):
        train_model(model, train_dataloader, loss_fn, optimizer, device)
    
    # Create the DataLoader for your validation data
    # Replace this with your DataLoader implementation
    val_dataloader = DataLoader(...)
    
    # Evaluate the model
    val_loss = evaluate_model(model, val_dataloader, loss_fn, device)
    
    # Store the results
    results.append({"params": params, "val_loss": val_loss})

# Find the best hyperparameters
best_result = min(results, key=lambda x: x["val_loss"])
print(f"Best hyperparameters: {best_result['params']}")
print(f"Validation loss: {best_result['val_loss']}")